<a href="https://colab.research.google.com/github/visumania/DeepSexist/blob/main/Dataset_Task_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Montura de google colab para poder leer/escribir en Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importación de librerías necesarias

In [ ]:
import pandas as pd

Tratamiento del fichero orifinal para generar uno nuevo en donde solamente se recoja la información relevante para la tarea 3.3

Etiquetas a tratar:
- **IDEOLOGICAL AND INEQUALITY**: el texto desacredita el movimiento feminista, rechaza la desigualdad entre hombres y mujeres o presente a los hombres como víctimas de la opresión del género.
- **STEREOTYPING AND DOMINANCE**: el texto expresa ideas falsas sobre las mujeres que sugieren que son más adecuadas para desempeñar ciertos papeles (madre, esposa, cuidadora familiar, fiel, tierna, cariñosa, sumisa, etc.), o inadecuadas para ciertas tareas (conducir, trabajo duro, etc.), o afirmar que los hombres son de alguna manera superiores a las mujeres.
- **OBJECTIFICATION**: el texto presenta a las mujeres como objetos al margen de su dignidad y aspectos personales, o asume o describe determinadas cualidades físicas que deben tener las mujeres para cumplir los roles tradicionales de género (cumplimiento de cánones de belleza, hipersexualización de los atributos femeninos, cuerpos de las mujeres a disposición de los hombres, etc.).
- **SEXUAL VIOLENCE**: se hacen sugerencias sexuales, se piden favores sexuales o se ejerce el acoso de naturaleza sexual (violación o agresión sexual).
- **MISOGYNY AND NON-SEXUAL VIOLENCE**: el texto expresa odio y violencia hacia las mujeres.

Para esta tarea de clasificación multi-etiqueta, vamos a proponer trabajar con 3 ficheros diferentes (extraídos a partir del fichero original base proporcionado por la competición) los cuáles abarcan los conflictos de 3 maneras diferentes:
- **EXIST2025_training_task3_3_ALL.csv**: el etiquetado final se hará siguiendo el comportamiento de la operación lógica *OR* con el etiquetado de cada uno de los anotadores
- **EXIST2025_training_task3_3_MEN.csv**: Prevace solamente lo que etiquete el Hombre, descartando lo que digan las mujeres en la anotación.En caso de tener conflicto con 2 anotadores (Hombre y mujer) prevalecerá la anotación del hombre.
- **EXIST2025_training_task3_3_WOMEN.csv**: Prevalece solamente lo que etiquete la Mujer, descartando lo que digan los hombres en la anotación. En caso de tener conflicto con 2 anotadores (Hombre y mujer) prevalecerá la anotación de la mujer.


Lectura y tratamiento de los datos del fichero

In [ ]:
df = pd.read_json('/content/drive/MyDrive/dataset/EXIST 2025 Videos Dataset/training/EXIST2025_training.json', orient='index') # Lectura del dataset desde Drive
df.reset_index(drop=True, inplace=True) # Reseteamos el index del dataframe
df = df.drop(columns=["id_Tiktok", "video", "url", "annotators", "labels_task3_2", "split"]) # Eliminación de columnas que no son relevantes del dataframe

In [ ]:
# Eliminamos la columna correspondiente a la tarea 3.1
df_3_3 = df.drop(columns=["labels_task3_1"])

# Expande la columna labels_task3_3 y gender_annotators en varias columnas en varias columnas
df_3_3_expanded = df_3_3["labels_task3_3"].apply(pd.Series)
df_3_3_expanded_gender = df_3_3["gender_annotators"].apply(pd.Series)

# Renombra las columnas resultantes para mayor claridad
df_3_3_expanded.columns = [f"labels_task_3_3_{i+1}" for i in range(df_3_3_expanded.shape[1])]
df_3_3_expanded_gender.columns = [f"gender_annotators_{i+1}" for i in range(df_3_3_expanded_gender.shape[1])]

print(df_3_3['gender_annotators'].value_counts(),"\n")

# Une con el dataframe original sin la columna original
df_3_3 = pd.concat([df_3_3.drop(columns=["labels_task3_3"]), df_3_3_expanded], axis=1)
df_3_3 = pd.concat([df_3_3.drop(columns=["gender_annotators"]), df_3_3_expanded_gender], axis=1)

# Reordenamos el orden las columnas para una mejor visión del dataframe
df_3_3 = df_3_3[["lang", "id_EXIST", "text", "path_video", "number_annotators", "gender_annotators_1", "gender_annotators_2", "gender_annotators_3", "labels_task_3_3_1", "labels_task_3_3_2", "labels_task_3_3_3"]]

# Creación de las nuevas columnas correspondientes a las etiquetas de la tarea 3_3
df_3_3[["IDEOLOGICAL-INEQUALITY", "STEREOTYPING-DOMINANCE", "OBJECTIFICATION", "SEXUAL-VIOLENCE", "MISOGYNY-NON-SEXUAL-VIOLENCE"]] = 0

gender_annotators
[F, M]       1438
[F, M, M]     428
[F, F]        342
[F, M, F]     166
[F, F, F]     150
Name: count, dtype: int64 



Construcción del fichero **EXIST2025_training_task3_3_ALL.csv**

In [ ]:
for i in range(df_3_3.shape[0]):
  # Para labels_task_3_3_1
  if "IDEOLOGICAL-INEQUALITY" in df_3_3.iloc[i]['labels_task_3_3_1']:
    df_3_3.at[i, "IDEOLOGICAL-INEQUALITY"] = 1
  if "STEREOTYPING-DOMINANCE" in df_3_3.iloc[i]['labels_task_3_3_1']:
    df_3_3.at[i, "STEREOTYPING-DOMINANCE"] = 1
  if "OBJECTIFICATION" in df_3_3.iloc[i]['labels_task_3_3_1']:
    df_3_3.at[i, "OBJECTIFICATION"] = 1
  if "SEXUAL-VIOLENCE" in df_3_3.iloc[i]['labels_task_3_3_1']:
    df_3_3.at[i, "SEXUAL-VIOLENCE"] = 1
  if "MISOGYNY-NON-SEXUAL-VIOLENCE" in df_3_3.iloc[i]['labels_task_3_3_1']:
    df_3_3.at[i, "MISOGYNY-NON-SEXUAL-VIOLENCE"] = 1

  # Para labels_task_3_3_2
  if "IDEOLOGICAL-INEQUALITY" in df_3_3.iloc[i]['labels_task_3_3_2']:
    df_3_3.at[i, "IDEOLOGICAL-INEQUALITY"] = 1
  if "STEREOTYPING-DOMINANCE" in df_3_3.iloc[i]['labels_task_3_3_2']:
    df_3_3.at[i, "STEREOTYPING-DOMINANCE"] = 1
  if "OBJECTIFICATION" in df_3_3.iloc[i]['labels_task_3_3_2']:
    df_3_3.at[i, "OBJECTIFICATION"] = 1
  if "SEXUAL-VIOLENCE" in df_3_3.iloc[i]['labels_task_3_3_2']:
    df_3_3.at[i, "SEXUAL-VIOLENCE"] = 1
  if "MISOGYNY-NON-SEXUAL-VIOLENCE" in df_3_3.iloc[i]['labels_task_3_3_2']:
    df_3_3.at[i, "MISOGYNY-NON-SEXUAL-VIOLENCE"] = 1

  # Para labels_task_3_3_3 --> Tenemos que tener en cuenta los valores NaN
  if df_3_3.iloc[i]['number_annotators'] == 3: # Valores que no van a tener NaN
    if "IDEOLOGICAL-INEQUALITY" in df_3_3.iloc[i]['labels_task_3_3_3']:
      df_3_3.at[i, "IDEOLOGICAL-INEQUALITY"] = 1
    if "STEREOTYPING-DOMINANCE" in df_3_3.iloc[i]['labels_task_3_3_3']:
      df_3_3.at[i, "STEREOTYPING-DOMINANCE"] = 1
    if "OBJECTIFICATION" in df_3_3.iloc[i]['labels_task_3_3_3']:
      df_3_3.at[i, "OBJECTIFICATION"] = 1
    if "SEXUAL-VIOLENCE" in df_3_3.iloc[i]['labels_task_3_3_3']:
      df_3_3.at[i, "SEXUAL-VIOLENCE"] = 1
    if "MISOGYNY-NON-SEXUAL-VIOLENCE" in df_3_3.iloc[i]['labels_task_3_3_3']:
      df_3_3.at[i, "MISOGYNY-NON-SEXUAL-VIOLENCE"] = 1



In [ ]:
# Eliminación de columnas que no son relevantes del dataframe
df_3_3 = df_3_3.drop(columns=["number_annotators", "gender_annotators_1", "gender_annotators_2", "gender_annotators_3", "labels_task_3_3_1", "labels_task_3_3_2", "labels_task_3_3_3"])

In [ ]:
# Guardamos el fichero resultante en nuestro Drive
df_3_3.T.to_csv('/content/drive/MyDrive/dataset/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_3_ALL.csv')